# DAT500 - FinalProject, Flight Delay Prediction

Group Members:

    - Brage Solheim
    - Yohannes D. Kassaye

## Introduction

So far we have used map reduce jobs to find null values in our data, find mean and mode of each column, and fill null values with mean or mode.

We will be predicting flight delays based on data that is available prior to the day of the flight. <br>
Such data could be weather, airport, airline, origin and destination, and other factors.

## Initlization

In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark'

In [2]:
from datetime import datetime, date
import pandas as pd
import numpy as np
from pyspark.sql import Row
from pyspark.sql import SparkSession
import pyspark.pandas as ps

In [3]:
ORIGINAL_DATA_PATH = "hdfs:///data/airline_delay_data.csv"
CLEANED_DATA_PATH = "hdfs:///data/airline_delay_data_cleaned.csv"

TEST_DATA_ORIGINAL_PATH = "/data/2015.sample.csv"
TEST_DATA_CLEANED_PATH = "/data/2015.sample.cleaned.csv"

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import pyspark
conf = pyspark.SparkConf().setAll([("spark.driver.maxResultSize", '4g'), ("spark.sql.execution.arrow.pyspark.enabled", 'true')])

In [6]:
spark = (SparkSession
            .builder
            .master("yarn") # for cluster
            # .master("local[*]") # for local testing
            .config(conf=conf)
            .appName("Flight-delay-prediction")
            .getOrCreate())

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-21 19:05:38,076 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-21 19:05:46,285 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [7]:
print(sc.getConf().getAll())

[('spark.sql.warehouse.dir', 'file:/home/ubuntu/DAT500-FinalProject/spark/spark-warehouse'), ('spark.executor.id', 'driver'), ('spark.app.name', 'Flight-delay-prediction'), ('spark.app.startTime', '1650567937562'), ('spark.driver.port', '41669'), ('spark.driver.host', 'namenode'), ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES', 'http://namenode:8088/proxy/application_1645016454320_0107'), ('spark.master', 'yarn'), ('spark.executorEnv.PYTHONPATH', '/usr/local/spark/python/:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.9.3-src.zip'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.ui.proxyBase', '/proxy/application_1645016454320_0107'), ('spark.submit.pyFiles', ''), ('spark.yarn.isPython', 'true'), ('spark.submit.deployMode', 'client'), ('spark.app.id', 'application_1645016454320_0107'), ('spark.ui.filters', 'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'), ('spark.org.apache.hadoop.yarn.serve

In [8]:
from pyspark.sql.types import StructType, StructField, StringType, DateType, DoubleType

In [9]:
features = ["FL_DATE","OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST","CRS_DEP_TIME","DEP_TIME","DEP_DELAY","TAXI_OUT","WHEELS_OFF","WHEELS_ON","TAXI_IN","CRS_ARR_TIME","ARR_TIME","ARR_DELAY","CANCELLED","DIVERTED","CRS_ELAPSED_TIME","ACTUAL_ELAPSED_TIME","AIR_TIME","DISTANCE","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]
nullable=False
schema = StructType([
    StructField("FL_DATE", DateType(), nullable), StructField("OP_CARRIER", StringType(), nullable), StructField("OP_CARRIER_FL_NUM", DoubleType(), nullable),
    StructField("ORIGIN", StringType(), nullable), StructField("DEST", StringType(), nullable), StructField("CRS_DEP_TIME", DoubleType(), nullable),
    StructField("DEP_TIME", DoubleType(), nullable), StructField("DEP_DELAY", DoubleType(), nullable), StructField("TAXI_OUT", DoubleType(), nullable),
    StructField("WHEELS_OFF", DoubleType(), nullable), StructField("WHEELS_ON", DoubleType(), nullable), StructField("TAXI_IN", DoubleType(), nullable),
    StructField("CRS_ARR_TIME", DoubleType(), nullable), StructField("ARR_TIME", DoubleType(), nullable), StructField("ARR_DELAY", DoubleType(), nullable),
    StructField("CANCELLED", DoubleType(), nullable), StructField("DIVERTED", DoubleType(), nullable), StructField("CRS_ELAPSED_TIME", DoubleType(), nullable),
    StructField("ACTUAL_ELAPSED_TIME", DoubleType(), nullable), StructField("AIR_TIME", DoubleType(), nullable), StructField("DISTANCE", DoubleType(), nullable),
    StructField("CARRIER_DELAY", DoubleType(), nullable), StructField("WEATHER_DELAY", DoubleType(), nullable), StructField("NAS_DELAY", DoubleType(), nullable),
    StructField("SECURITY_DELAY", DoubleType(), nullable), StructField("LATE_AIRCRAFT_DELAY", DoubleType(), nullable)])

In [10]:
df = spark.read.csv(CLEANED_DATA_PATH, header=True, schema=schema, inferSchema=False)
df.to_pandas_on_spark().head()

2022-04-21 19:06:16,309 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 19:06:16,362 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 19:06:16,402 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2016-09-25,AA,2622.0,EGE,DFW,800.0,749.0,-11.0,11.0,800.0,1049.0,15.0,1110.0,1104.0,-6.0,0.0,0.0,130.0,135.0,109.0,721.0,19.62,3.04,15.08,0.09,24.72
1,2016-09-25,AA,2623.0,DFW,TPA,1830.0,1915.0,45.0,20.0,1935.0,2243.0,3.0,2155.0,2246.0,51.0,0.0,0.0,145.0,151.0,128.0,929.0,0.00,0.00,6.00,0.00,45.00
2,2016-09-25,AA,2625.0,MCO,DFW,935.0,930.0,-5.0,12.0,942.0,1052.0,17.0,1124.0,1109.0,-15.0,0.0,0.0,169.0,159.0,130.0,985.0,19.62,3.04,15.08,0.09,24.72
3,2016-09-25,AA,2626.0,DFW,FLL,1429.0,1710.0,161.0,28.0,1738.0,2054.0,4.0,1814.0,2058.0,164.0,0.0,0.0,165.0,168.0,136.0,1119.0,15.00,0.00,3.00,0.00,146.00
4,2016-09-25,AA,2626.0,TUS,DFW,915.0,910.0,-5.0,10.0,920.0,1337.0,14.0,1328.0,1351.0,23.0,0.0,0.0,133.0,161.0,137.0,813.0,0.00,0.00,23.00,0.00,0.00


## Data Engineering

In [11]:
df = df.repartition(24)

In [12]:
from pyspark.sql import functions as F

As we see the order of the data is completely mixed up, so our header is in the middle of the data. <br>
We will have to find that header row, and then remove it from the data.

In [13]:
header_row = df.filter(F.col("FL_DATE").isNull())
header_row.to_pandas_on_spark()

2022-04-21 19:07:58,323 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 19:07:58,324 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 19:08:47,086 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,None,OP_CARRIER,NaN,ORIGIN,DEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


When we infer the schema, the header row is not the same type as the rest of the rows, and therefore is converted to null. <br>
And we can use this fact to filter out the header row.

In [14]:
df = df.filter((F.col("FL_DATE").isNull() & F.col("OP_CARRIER").isNotNull() & F.col("ORIGIN").isNotNull() & F.col("DEST").isNotNull()) == False)

In [15]:
header_row = df.filter(F.col("FL_DATE").isNull())
print(header_row.count())

0


Next we will encode categorical columns, because we will be using them in our model.

In [16]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

categorical_columns = [col[0] for col in df.dtypes if col[1] == "string"]
indexers = [StringIndexer(inputCol=col, outputCol=col+"_indexed", handleInvalid="keep").fit(df) for col in categorical_columns]

pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df).transform(df)
df_indexed.show(5)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+------------------+--------------+------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|OP_CARRIER_indexed|ORIGIN_indexed|DEST_indexed|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+------------------

We are also dealing with time series data, and therefore we need to encode the dates.

In [17]:
df_indexed = df_indexed.withColumn("FL_DATE", F.col("FL_DATE").cast("timestamp"))\
             .withColumn("FL_DATE_indexed", F.unix_timestamp(F.col("FL_DATE"), format="yyyy-MM-dd"))
df_indexed.show(5)

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+------------------+--------------+------------+---------------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|OP_CARRIER_indexed|ORIGIN_indexed|DEST_indexed|FL_DATE_indexed|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+------------

To make sure that our model understand the columns containing data such as Planned Departure Time, and Planned Arrival Time, we will convert them to time.

In [18]:
def Convert_time_to_string(time, date):

    time = str(int(time))
    if len(time) == 1:
        time = f"00:0{time[0]}"
    elif len(time) == 2:
        time = f"00:{time}"
    elif len(time) == 3:
        time = f"0{time[0]}:{time[1:]}"
    elif len(time) == 4:
        time = f"{time[0:2]}:{time[2:]}"
    
    if date != None:
        return f"{date} {time}"

    return time

udf_Convert_time_to_string = F.udf(Convert_time_to_string, StringType())

In [19]:
print(Convert_time_to_string(0.0, "2018-11-04"), # should be 2018-11-04 00:00
      Convert_time_to_string(4.0, "2018-11-04"), # should be  2018-11-04 00:04, because you cant have leading zeros
      Convert_time_to_string(23.0, "2018-11-04"), # should be  2018-11-04 00:23, because you cant have leading zeros
      Convert_time_to_string(808.0, "2018-11-04"), # should be  2018-11-04 08:08, because you cant have leading zeros
      Convert_time_to_string(1203.0, "2018-11-04")) # should be  2018-11-04 12:03

2018-11-04 00:00 2018-11-04 00:04 2018-11-04 00:23 2018-11-04 08:08 2018-11-04 12:03


In [20]:
df_indexed.withColumn("FL_DATE", F.split(F.col("FL_DATE").cast("string"), " ").getItem(0) + "hei").show(5)

+-------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+------------------+--------------+------------+---------------+
|FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|OP_CARRIER_indexed|ORIGIN_indexed|DEST_indexed|FL_DATE_indexed|
+-------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+----------

In [21]:
df_indexed = df_indexed.withColumn("CRS_DEP_TIME_indexed", F.unix_timestamp(udf_Convert_time_to_string(F.col("CRS_DEP_TIME"), F.split(F.col("FL_DATE").cast("string"), " ").getItem(0)), format="yyyy-MM-dd HH:mm")) \
    .withColumn("CRS_ARR_TIME_indexed", F.unix_timestamp(udf_Convert_time_to_string(F.col("CRS_ARR_TIME"), F.split(F.col("FL_DATE").cast("string"), " ").getItem(0)), format="yyyy-MM-dd HH:mm"))
df_indexed.show(5, truncate=False)

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+------------------+--------------+------------+---------------+--------------------+--------------------+
|FL_DATE            |OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|OP_CARRIER_indexed|ORIGIN_indexed|DEST_indexed|FL_DATE_indexed|CRS_DEP_TIME_indexed|CRS_ARR_TIME_indexed|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+--

We will try to predict whether or not the flight is delayed, so we will create a binary column, based on either departure delay or arrival delay.

In [22]:
df_indexed = df_indexed.withColumn("Delayed", F.when((F.col("DEP_DELAY") > 0) | (F.col("ARR_DELAY") > 0), 1).otherwise(0))
df_indexed.show(5, truncate=False)

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+------------------+--------------+------------+---------------+--------------------+--------------------+-------+
|FL_DATE            |OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|OP_CARRIER_indexed|ORIGIN_indexed|DEST_indexed|FL_DATE_indexed|CRS_DEP_TIME_indexed|CRS_ARR_TIME_indexed|Delayed|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+-----

We also have some data regarding a flight being canceled, so we will remove any rows that have a value of 1 in the Cancelled column. <br>
given that it doesnt give us any information about the flight being delayed.

In [23]:
df_indexed.groupBy("CANCELLED").count().show()

+---------+--------+
|CANCELLED|   count|
+---------+--------+
|      0.0|23969781|
|      1.0|  355022|
+---------+--------+



In [24]:
df_indexed = df_indexed.where(F.col("CANCELLED") == 0)
df_indexed.show(5, truncate=False)

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+------------------+--------------+------------+---------------+--------------------+--------------------+-------+
|FL_DATE            |OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|OP_CARRIER_indexed|ORIGIN_indexed|DEST_indexed|FL_DATE_indexed|CRS_DEP_TIME_indexed|CRS_ARR_TIME_indexed|Delayed|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+-----

Lastly we will remove columns containing data that is not available prior to the day of the flight. <br>

These columns are:

    - DEP_TIME - Actual Departure Time
    - DEP_DELAY - Total Delay on Departure in minutes
    - TAXI_OUT - Time between departure The time duration elapsed between departure from the origin airport gate and wheels off
    - WHEELS_OFF - The time point that the aircraft's wheels leave the ground.
    - WHEELS_ON - The time point that the aircraft'ss wheels touch on the ground.
    - TAXI_IN - The time duration elapsed between wheels-on and gate arrival at the destination airport.
    - ARR_TIME = Actual Arrival Time = ARRIVAL_TIME - SCHEDULED_ARRIVAL
    - ARR_DELAY = Total Delay on Arrival in minutes
    - DIVERTED = Aircraft landed on different airport that the one scheduled
    - AIR_TIME = The time duration between wheels_off and wheels_on time
    - DISTANCE = Distance between two airports
    - CARRIER_DELAY = Delay caused by the airline in minutes
    - NAS_DELAY = Delay caused by air system
    - SECURITY_DELAY = caused by security reasons
    - LATE_AIRCRAFT_DELAY = Delay caused by security

In [25]:
cols_to_remove = ["DEP_TIME", "DEP_DELAY", "TAXI_OUT", "WHEELS_OFF", "WHEELS_ON", "TAXI_IN", "ARR_TIME", "ARR_DELAY", "DIVERTED", "AIR_TIME", "DISTANCE", "CARRIER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY"]
df_indexed = df_indexed.drop(*cols_to_remove)

In [26]:
df_indexed.to_pandas_on_spark().head()

2022-04-21 19:26:10,290 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 19:26:10,292 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 19:27:43,273 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,CRS_ARR_TIME,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,WEATHER_DELAY,OP_CARRIER_indexed,ORIGIN_indexed,DEST_indexed,FL_DATE_indexed,CRS_DEP_TIME_indexed,CRS_ARR_TIME_indexed,Delayed
0,2016-09-21,DL,1998.0,MSP,GRR,1526.0,1745.0,0.0,79.0,94.0,3.04,1.0,10.0,77.0,1474416000,1474471560,1474479900,1
1,2018-11-10,YV,5819.0,ABQ,PHX,615.0,744.0,0.0,89.0,82.0,3.04,16.0,56.0,6.0,1541808000,1541830500,1541835840,0
2,2018-08-19,OH,5377.0,ORD,BDL,715.0,1019.0,0.0,124.0,123.0,3.04,13.0,1.0,54.0,1534636800,1534662900,1534673940,0
3,2018-08-13,OO,2983.0,GJT,PHX,1510.0,1556.0,0.0,106.0,106.0,0.00,3.0,150.0,6.0,1534118400,1534173000,1534175760,1
4,2016-09-22,AA,266.0,LIH,LAX,1425.0,2306.0,0.0,341.0,333.0,0.00,2.0,76.0,4.0,1474502400,1474554300,1474585560,1


## Data Exploration and Analysis

So far we have cleaned the data by removing and filling null values.
Now we will try to find the correlation between the our target, DEP_DELAY, and the other columns.

In [27]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

In [28]:
def GetCorrelationMatrix(df):
    inputcols = [col[0] for col in df_indexed.dtypes if col[1] == "double" or col[1] == "int" or col[1] == "bigint" or col[1] == "long"]
    vector_col = "features"

    assembler = VectorAssembler(inputCols=inputcols, outputCol=vector_col, handleInvalid="skip")
    df_vector = assembler.transform(df).select(vector_col)
    matrix = Correlation.corr(df_vector, "features")

    matrix_df = spark.createDataFrame(matrix.collect()[0]["pearson({})".format(vector_col)].toArray().tolist(), inputcols).to_pandas_on_spark()
    matrix_df["feature"] = inputcols

    return matrix_df
    

In [29]:
corr_matrix = GetCorrelationMatrix(df_indexed)

2022-04-21 19:33:44,067 WARN correlation.PearsonCorrelation: Pearson correlation matrix contains NaN values.


In [30]:
corr_matrix.set_index("feature")

,OP_CARRIER_FL_NUM,CRS_DEP_TIME,CRS_ARR_TIME,CANCELLED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,WEATHER_DELAY,OP_CARRIER_indexed,ORIGIN_indexed,DEST_indexed,FL_DATE_indexed,CRS_DEP_TIME_indexed,CRS_ARR_TIME_indexed,Delayed
feature,,,,,,,,,,,,,,
OP_CARRIER_FL_NUM,1.000000,0.003467,0.001306,NaN,-0.302706,-0.296747,0.005099,0.079983,0.220909,0.225674,0.093118,0.093120,0.093119,-0.021921
CRS_DEP_TIME,0.003467,1.000000,0.682710,NaN,-0.015373,-0.017952,-0.000974,0.001410,-0.121237,0.129927,-0.000929,-0.000457,-0.000590,0.143310
CRS_ARR_TIME,0.001306,0.682710,1.000000,NaN,0.026066,0.025037,0.000123,-0.003089,-0.113905,0.082688,-0.006736,-0.006413,-0.006239,0.128429
CANCELLED,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CRS_ELAPSED_TIME,-0.302706,-0.015373,0.026066,NaN,1.000000,0.983058,-0.002673,0.011934,-0.205367,-0.232022,-0.002568,-0.002575,-0.002555,0.059892
ACTUAL_ELAPSED_TIME,-0.296747,-0.017952,0.025037,NaN,0.983058,1.000000,-0.000726,0.024770,-0.199206,-0.226301,-0.002379,-0.002387,-0.002367,0.112584
WEATHER_DELAY,0.005099,-0.000974,0.000123,NaN,-0.002673,-0.000726,1.000000,0.000602,0.002425,0.007813,0.004097,0.004096,0.004097,0.000083
OP_CARRIER_indexed,0.079983,0.001410,-0.003089,NaN,0.011934,0.024770,0.000602,1.000000,0.119881,0.120088,0.100058,0.100059,0.100057,-0.022266
ORIGIN_indexed,0.220909,-0.121237,-0.113905,NaN,-0.205367,-0.199206,0.002425,0.119881,1.000000,-0.164176,0.023812,0.023754,0.023755,-0.061795


In [31]:
DELAY_CORR = corr_matrix[["Delayed", "feature"]].sort_values(by="Delayed", ascending=False)
DELAY_CORR

2022-04-21 19:33:49,931 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 19:33:50,081 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,Delayed,feature
3,NaN,CANCELLED
13,1.000000,Delayed
1,0.143310,CRS_DEP_TIME
2,0.128429,CRS_ARR_TIME
5,0.112584,ACTUAL_ELAPSED_TIME
4,0.059892,CRS_ELAPSED_TIME
6,0.000083,WEATHER_DELAY
11,-0.017215,CRS_DEP_TIME_indexed
12,-0.017219,CRS_ARR_TIME_indexed
10,-0.017283,FL_DATE_indexed


We see that there are some features with negative correlations with respect to our target feature, so we will remove those.

High degree: If the coefficient value lies between ± 0.50 and ± 1, then it is said to be a strong correlation. Moderate degree: If the value lies between ± 0.30 and ± 0.49, then it is said to be a medium correlation. Low degree: When the value lies below + . 29, then it is said to be a small correlation.

Need to figure out if using negative treshold for correlation is a good idea. It looks like the features affecting DEP_DELAY is the ones over

----

Further data analysis....

----

## What will we predict?

We will be predicting flight delays based on data that is available prior to the day of the flight. <br>
Such data could be weather, airport, airline, origin and destination, and other factors.

## TODOs:
- [ ] - Select a regression model, and start predicting.
- [ ] - If we are done early with that we can try to optimize the time spark / hadoop - mapreduce tasks take.
        - [ ] - repartition the data to use right amount cores. (24 cores)